In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_rows",40)
%matplotlib inline

In [2]:
class dataset:
    kdd_train_2labels = pd.read_pickle("dataset/kdd_train_2labels.pkl")
    kdd_train_2labels_y = pd.read_pickle("dataset/kdd_train_2labels_y.pkl")
    
    kdd_test_2labels = pd.read_pickle("dataset/kdd_test_2labels.pkl")
    kdd_test_2labels_y = pd.read_pickle("dataset/kdd_test_2labels_y.pkl")
    

In [3]:
from sklearn.preprocessing import LabelEncoder

le_2labels = LabelEncoder()
dataset.y_train_2labels = le_2labels.fit_transform(dataset.kdd_train_2labels_y)
dataset.y_test_2labels = le_2labels.transform(dataset.kdd_test_2labels_y)


In [4]:
from itertools import product
from sklearn.model_selection import train_test_split

class preprocessing:
    x_train = dataset.kdd_train_2labels.iloc[:,:-2].values
    y_train = np.array(dataset.y_train_2labels)

    x_test, y_test = (dataset.kdd_test_2labels.iloc[:,:-2].values, 
                      np.array(dataset.y_test_2labels))

    

In [ ]:
from collections import namedtuple
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras import regularizers

class Train:
    score = namedtuple("score", ['epoch', 'no_of_features','hidden_layers','train_score', 'test_score'])
    #model_detail = namedtuple("model_detail", ['epoch', 'no_of_features','hidden_layers', 'model'])
    scores = []
    predictions = pd.DataFrame()
    #models = []
    def execute(x_train, x_test, 
                y_train, y_test, 
                input_dim, no_of_features, hidden_layers,
                epochs = 1, keep_prob = 0.4):
        
        print("Training for no_of_features: {}, hidden_layer: {}".format(no_of_features, hidden_layers
                                                                        ))
        model = Sequential()
        model.add(Dense(no_of_features, input_dim=input_dim, activation='relu'))
        model.add(Dropout(keep_prob))
        model.add(BatchNormalization())
        
        for i in range(hidden_layers - 1):
            model.add(Dense(no_of_features, activation='relu'))
            model.add(Dropout(keep_prob))
            model.add(BatchNormalization())

        
        model.add(Dense(1, activation=None))

        model.compile(loss='mean_squared_error',
                      optimizer="Adam",
                      metrics=['accuracy'])

        x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=.6)
        
        model.fit(x_train, y_train,
                  validation_data=(x_valid, y_valid),
                  epochs=epochs,
                  batch_size=128,
                  verbose = 0)
        
        curr_score_valid = model.evaluate(x_valid, y_valid) #, batch_size=128)
        curr_score_test = model.evaluate(x_test, y_test) #, batch_size=128)
        pred_value = model.predict(x_test)
        
        print("\n Train Accuracy: {}, Test Accuracy: {}".format(curr_score_valid[1], curr_score_test[1])  )
        Train.scores.append(Train.score(epochs,no_of_features,hidden_layers,curr_score_valid[1], curr_score_test[1]))
        #Train.models.append(Train.model_detail(epochs,no_of_features,hidden_layers,model))
        y_pred = pred_value[:,-1]
        y_pred[y_pred >= pred_value[:,-1].mean()] = 1
        y_pred[y_pred < pred_value[:,-1].mean()] = 0
        curr_pred = pd.DataFrame({"{}_{}_{}".format(epochs,f,h):y_pred},)
        Train.predictions = pd.concat([Train.predictions, curr_pred], axis = 1)
                

Using TensorFlow backend.


In [ ]:
features_arr = [4, 8, 16, 32, 64, 128, 256, 1024]
hidden_layers_arr = [2, 4, 6, 50, 100]


for f, h in product(features_arr, hidden_layers_arr):
    Train.execute(preprocessing.x_train, preprocessing.x_test, 
                  preprocessing.y_train, preprocessing.y_test, 
                 122, f, h)

Training for no_of_features: 4, hidden_layer: 2
21856/22544 [============================>.] - ETA: 0s
 Train Accuracy: 0.5343723539373413, Test Accuracy: 0.4308463449254791
Training for no_of_features: 4, hidden_layer: 4
22112/22544 [============================>.] - ETA: 0s
 Train Accuracy: 0.4642516934801016, Test Accuracy: 0.5489265436479773
Training for no_of_features: 4, hidden_layer: 6
22176/22544 [============================>.] - ETA: 0s
 Train Accuracy: 0.5344252751905165, Test Accuracy: 0.43075762952448543
Training for no_of_features: 4, hidden_layer: 50
22336/22544 [============================>.] - ETA: 0s
 Train Accuracy: 0.46413262066045724, Test Accuracy: 0.5692423704755145
Training for no_of_features: 4, hidden_layer: 100
22528/22544 [============================>.] - ETA: 0s
 Train Accuracy: 0.5346369602032176, Test Accuracy: 0.43075762952448543
Training for no_of_features: 8, hidden_layer: 2
21856/22544 [============================>.] - ETA: 0s
 Train Accuracy: 0.46

In [ ]:
pd.DataFrame(Train.scores)

In [ ]:
#for m in Train.models:
#    m.model.save("dataset/keras_model_epoch_{}_no_of_features_{}_hidden_layers_{}".format(m.epoch,
#                                                                                         m.no_of_features,
#                                                                                         m.hidden_layers))

In [ ]:
Train.predictions.to_pickle("dataset/dense_only_predictions.pkl")
pd.DataFrame(Train.scores).to_pickle("dataset/dense_only_scores.pkl")